In [1]:
import torch
import math
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
from torch.optim import Adam

import lightning as L
from torch.utils.data import TensorDataset, DataLoader

In [2]:
from CurrencyDataModule import CurrencyDataModule
dm = CurrencyDataModule('../cleaned/usdrub_new.csv', seq_length=5, columns=['close'], batch_size=16)

In [4]:
class LSTM(nn.Module):
  def __init__(self):
    super().__init__()

    self.lstm = nn.LSTM(input_size=1, hidden_size=1, batch_first=True)
    self.linear = nn.Linear(in_features=1, out_features=1)

  def forward(self, input):
    lstm_output = self.lstm(input)
    
    # [batch_size, 1]
    last_unrolled_values = lstm_output[0][:, -1, :]
    prediction = self.linear(last_unrolled_values)
    return prediction

In [ ]:
class LitLSTM(L.LightningModule):
  def __init__(self, lstm, learning_rate):
    super().__init__()
    self.lstm = lstm
    self.learning_rate = learning_rate

  def configure_optimizers(self):
    return Adam(self.parameters(), lr= self.learning_rate)
  
  def training_step(self, batch, batch_idx):
    input = batch[0]
    label = batch[1]
    # [batch_size, 1]
    output = self.lstm(input)
    loss = (output - label).pow(2).sum()

    self.log("train_loss", loss)
    return loss
  
  def test_step(self, batch, batch_idx):
    input = batch[0]
    label = batch[1]
    # [batch_size, 1]
    output = self.lstm(input)
    loss = (output - label).pow(2).sum()

    self.log("test_loss", loss)
    return loss